In [13]:
%pip install pandas matplotlib requests tqdm openpyxl



Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
import pandas as pd
import time
import logging
from tqdm import tqdm
import os
import json
import matplotlib.pyplot as plt

# Basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_config():
    try:
        with open('config.json', 'r') as config_file:
            return json.load(config_file)
    except FileNotFoundError:
        logging.error("Configuration file 'config.json' not found.")
        raise FileNotFoundError("Configuration file 'config.json' not found.")

config = load_config()

def load_users(filename):
    return pd.read_excel(filename)

def load_questions(filename):
    with open(filename, "r") as f:
        return [line.strip() for line in f if line.strip()]

def login_user(user):
    login_url = config["login_api_url"]
    payload = {
        "username": user["username"],
        "password": user["password"]
    }
    try:
        response = requests.post(login_url, json=payload)
        response.raise_for_status()
        return response.json()['access_token']
    except requests.exceptions.RequestException as e:
        logging.error(f"Login error for user {user['name']}: {e}")
        raise Exception(f"Could not log in user {user['name']}")

def send_question(user, question, access_token):
    url = config["api_url"]
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }
    payload = {"user_id": str(user["user_id"]), "question": question}
    start = time.time()
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        return {"time": time.time() - start, "data": response.json(), "error": None}
    except requests.exceptions.RequestException as e:
        logging.error(f"Error sending question for user {user['name']}: {e}")
        return {"time": time.time() - start, "data": None, "error": str(e)}

def run_test():
    users = load_users(config["user_file"])
    questions = load_questions(config["question_file"])
    total_response_times = []
    total_errors = []

    for index, user in enumerate(tqdm(users.iterrows(), desc="Processing users"), start=1):
        user_data = user[1]
        access_token = login_user(user_data)
        user_response_times = []
        success_count = 0
        error_count = 0

        for question in questions:
            result = send_question(user_data.to_dict(), question, access_token)
            user_response_times.append(result["time"])
            if result["error"]:
                error_count += 1
                logging.info(f"User {user_data['name']} asked: '{question}', error: {result['error']}")
            else:
                success_count += 1
                logging.info(f"User {user_data['name']} asked: '{question}', received: {result['data'].get('answer')}")

        total_response_times.append(user_response_times)
        total_errors.append((success_count, error_count))
        plot_user_response_times(user_data, user_response_times, success_count, error_count)

    plt.figure(figsize=(12, 8))
    plt.boxplot(total_response_times, labels=[f"User {i}" for i in range(1, len(total_response_times) + 1)])
    plt.title('Response Times Across All Users')
    plt.xlabel('User Index')
    plt.ylabel('Response Time (s)')
    plt.show()

def plot_user_response_times(user_data, response_times, success_count, error_count):
    plt.figure(figsize=(12, 6))
    plt.hist(response_times, bins=20, alpha=0.75, label=f"User {user_data['name']}")
    plt.title(f"Response Times Distribution for {user_data['name']}")
    plt.xlabel('Response Time (s)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.subplot(122)
    plt.bar(['Successes', 'Errors'], [success_count, error_count], color=['green', 'red'])
    plt.title(f'Success/Error Counts for {user_data["name"]}')
    plt.ylabel('Count')
    plt.show()

#if __name__ == "__main__":
#    run_test()
